In [104]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from time import sleep
import random
from tqdm.auto import tqdm, trange
import pandas as pd

In [105]:
import matplotlib.pyplot as plt
import seaborn as sns

In [106]:
options = webdriver.ChromeOptions()
################################################### 본인 user-agent로 바꾸기
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36')

In [111]:
# 크롬 창 열기
################################################### 본인 chromedriver 위치로 바꾸기
driver = webdriver.Chrome("C:/Users/yello/mba-insight/chromedriver_win32/chromedriver.exe", options=options)
################################################### 원하는 url로 바꾸기
url = 'https://play.google.com/store/apps/details?id=com.wooribank.smart.npib'
driver.get(url)

<ipython-input-111-1dada62bbcc9>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/Users/yello/mba-insight/chromedriver_win32/chromedriver.exe", options=options)


In [112]:
# 페이지 로딩 대기
wait = WebDriverWait(driver, 10)

# '리뷰 모두 보기' 버튼 렌더링 확인
all_review_button_xpath = '/html/body/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[4]/section/div/div/div[5]/div/div/button/span'
button_loading_wait = wait.until(EC.presence_of_element_located((By.XPATH, all_review_button_xpath)))

# '리뷰 모두 보기' 버튼 클릭
driver.find_element_by_xpath(all_review_button_xpath).click()

# '리뷰 모두 보기' 페이지 렌더링 대기
all_review_page_xpath = '/html/body/div[4]/div[2]/div/div/div/div/div[2]'
page_loading_wait = wait.until(EC.presence_of_element_located((By.XPATH, all_review_page_xpath)))

<ipython-input-112-a605cd2d9883>:9: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(all_review_button_xpath).click()


In [113]:
import time

def scroll(modal):
    try:        
        # 스크롤 높이 받아오기
        last_height = driver.execute_script("return arguments[0].scrollHeight", modal)
        iter = 0
        while True:
#        for i in range(10):
            pause_time = random.uniform(0.5, 0.8)
            # 최하단까지 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", modal)
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight-50);", modal)
            time.sleep(pause_time)
            # 스크롤 높이 새롭게 받아오기
            new_height = driver.execute_script("return arguments[0].scrollHeight", modal)
                     
            try:
                # '더보기' 버튼 있을 경우 클릭
                all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()
            except:
                # 스크롤 완료 경우
                if new_height == last_height:
                    print("스크롤 완료")
                    break
                last_height = new_height
            
            if iter % 10 == 0:
                time.sleep(20)
            iter += 1
                
    except Exception as e:
        print("에러 발생: ", e)

In [114]:
# 페이지 무한 스크롤 다운
modal = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='fysCi']")))
scroll(modal)

# # html parsing하기
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

# # html 데이터 저장
# with open("C:/Users/yello/mba/data_html.html", "w", encoding = 'utf-8') as file:
#     file.write(str(soup_source))
    
# 리뷰 데이터 클래스 접근
review_source = soup_source.find_all(class_ = 'RHo1pe')
# 리뷰 데이터 저장용 배열
dataset = []
# 데이터 넘버링을 위한 변수
review_num = 0
# 리뷰 1개씩 접근해 정보 추출
for review in tqdm(review_source):
    review_num+=1
    # 리뷰 등록일 데이터 추출
    date_full = review.find_all(class_ = 'bp9Aid')[0].text
    date_year = date_full[0:4] # 연도 데이터 추출
    # 해당 단어가 등장한 인덱스 추출
    year_index = date_full.find('년')
    month_index = date_full.find('월')
    day_index = date_full.find('일')
    
    date_month = str(int(date_full[year_index+1:month_index])) # 월(Month) 데이터 추출
    # 월 정보가 1자리의 경우 앞에 0 붙이기(e.g., 1월 -> 01월)
    if len(date_month) == 1:
        date_month = '0' + date_month
    
    date_day = str(int(date_full[month_index+1:day_index])) # 일(Day) 데이터 추출 
    # 일 정보가 1자리의 경우 앞에 0 붙여줌(e.g., 7일 -> 07일)
    if len(date_day) == 1:
        date_day = '0' + date_day
    
    # 리뷰 등록일 full version은 최종적으로 yyyymmdd 형태로 저장
    date_full = date_year + date_month + date_day
    user_name = review.find_all(class_ = 'X5PpBb')[0].text # 닉네임 데이터 추출
    rating = review.find_all(class_ = "iXRFPc")[0]['aria-label'][10] # 평점 데이터 추출
    content = review.find_all(class_ = 'h3YV2d')[0].text # 리뷰 데이터 추출

    data = {
        "id": review_num, 
        "date": date_full,
        "dateYear": date_year,
        "dateMonth": date_month,
        "dateDay": date_day,
        "rating": rating,
        "userName": user_name,
        "content": content
    }
    dataset.append(data)

<ipython-input-113-c0c161077044>:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()


스크롤 완료


In [115]:
df = pd.DataFrame(dataset)
################################################### 저장할 위치 바꾸기
# df.to_csv('C:/Users/yello/mba/wooribank_review_dataset.csv', encoding = 'utf-8-sig', index=False) # csv 파일로 저장

In [116]:
df.shape

(6000, 8)

In [35]:
df.head()

,id,date,dateYear,dateMonth,dateDay,rating,userName,content
0,1,20220418,2022,04,18,5,김흥서,[ W009474 ] 입출금알림을 원뱅킹앱으로 사용하면 연속적으로 출금이 됐을 때 ...
1,2,20220417,2022,04,17,5,so young kim,[ W009474 ] 확실히 간편해졌네요. 상품 가입도 쉽게 할 수 있어보이네요. ...
2,3,20220426,2022,04,26,1,이정신,우리은행에서 발급한 인증서를 가져오기해서 등록했지만 새로 로그인할때마다 등록된 인증...
3,4,20220311,2022,03,11,1,이소현,잘 될때는 잘 되지만 한 번 에러 발생하면 진짜 30분을 넘게 결제를 반복해서 진행...
4,5,20220303,2022,03,03,1,박경한,신분증 촬영이 안되네요. 초점이 안맞질않나 초점이 맞아서 제대로 맞춰도 아무반응이없...


In [37]:
print(df['id'].nunique())
print(len(df))

7400
7400


# Google-play-scraper 이용

In [39]:
!pip install google_play_scraper

  Created wheel for google-play-scraper: filename=google_play_scraper-1.0.5-py3-none-any.whl size=24473 sha256=bdc5b937c5e0658e7d05d8cab6e852e54f794f342edc9fd14b2827f06a93438d
  Stored in directory: c:\users\yello\appdata\local\pip\cache\wheels\7a\c5\f8\a45d4a7d6b7ce6d686a0576a7f7f02992dd7b3e70dc7d04fe6
Successfully built google-play-scraper


In [40]:
# from google_play_scraper import app

In [43]:
# result = app(
#     'com.wooribank.smart.npib',
#     lang='ko',
#     country='kr',
# )

In [47]:
# print(result)

{'title': [], 'description': None, 'descriptionHTML': None, 'summary': None, 'summaryHTML': None, 'installs': None, 'minInstalls': None, 'score': None, 'ratings': None, 'reviews': None, 'histogram': [0, 0, 0, 0, 0], 'price': None, 'free': None, 'currency': None, 'sale': False, 'saleTime': None, 'originalPrice': None, 'saleText': None, 'offersIAP': False, 'inAppProductPrice': None, 'size': [[None, [[1800], None, [72000]]]], 'androidVersion': None, 'androidVersionText': None, 'developer': None, 'developerId': None, 'developerEmail': None, 'developerWebsite': None, 'developerAddress': None, 'privacyPolicy': None, 'developerInternalID': None, 'genre': None, 'genreId': None, 'icon': None, 'headerImage': None, 'screenshots': [], 'video': None, 'videoImage': None, 'contentRating': None, 'contentRatingDescription': None, 'adSupported': None, 'containsAds': False, 'released': None, 'updated': None, 'version': [[['CBSqARUKEwjzvOyk7uH3AhVMvZYKHTAaD9s='], None, [], None, None, None, [None, '앱 평가하기

In [49]:
# from google_play_scraper import Sort, reviews_all

In [75]:
# result = reviews_all(
#     'com.wooribank.smart.npib',
#     sleep_milliseconds=0,
#     lang='ko',
#     country='kr',
#     sort=Sort.MOST_RELEVANT,
#     filter_score_with=0,
# )

TypeError: reviews() got an unexpected keyword argument 'num'

In [73]:
# len(result)

8952

In [77]:
from google_play_scraper import Sort, reviews

In [102]:
result = reviews(
    ################################# url에 id= 이후 부분 넣기
    'com.wooribank.smart.npib',
    lang='ko',
    country='kr',
    sort=Sort.MOST_RELEVANT,
    ################################# 불러오고자 하는 리뷰 개수 입력
    count=10000,
)

In [96]:
# print(result)

([{'reviewId': 'gp:AOqpTOGER-vDg9a3cue7lazCrWrGhHyQRIH2aMf5LKZu1Yc8LOHjszVexGXBjXm1XPTp4LP92gKJoNV9LLqvaA', 'userName': '김흥서', 'userImage': 'https://play-lh.googleusercontent.com/a/AATXAJy4gYc7m0tK6p89YSGLLR3WF-MPP2V3_wwWodwo=mo', 'content': '[ W009474 ] 입출금알림을 원뱅킹앱으로 사용하면 연속적으로 출금이 됐을 때 푸시가 잘 안 와요. 예를 들어 1일에 자동이체 (적금 대출이자 등)가 비슷한 시간대에 되면 4개 중에 첫번째 푸시 하나만 뜬다든지 아예 안 뜬다든지 하는 경우가 많네요. 입출금 푸시를 가계부 앱이랑 연동시켜놨는데 놓치는 경우가 많습니다. 개선되었으면 좋겠어요', 'score': 5, 'thumbsUpCount': 125, 'reviewCreatedVersion': '2.2.1', 'at': datetime.datetime(2022, 4, 18, 23, 11, 58), 'replyContent': None, 'repliedAt': None}, {'reviewId': 'gp:AOqpTOFxoSU_oPWTS3Jz-oUArpuhtgmko66Qtc4Ciqa9E5sCJ39TzkI9KqMgnvd7Gw7RmW-016T9yZZB2Pu8rw', 'userName': 'so young kim', 'userImage': 'https://play-lh.googleusercontent.com/a/AATXAJzo3Xnsd4PHQ7ikEJaIpZcwZZmB-ElMVkDdBuz1=mo', 'content': '[ W009474 ] 확실히 간편해졌네요. 상품 가입도 쉽게 할 수 있어보이네요. 한눈에 보기편하게 바뀌어서 좋은듯 합니다. UI도 되게 깔끔해져서 보기 좋고 예전에도 있었던 기능인지는 잘 모르겠는데 이체내역결과상세정보를 확인증으로 바로 저장해서 갤러리에서 확인도 할 수 있어

In [103]:
print(len(result[0]))

8953


In [133]:
df = pd.DataFrame()

df['date'] = list(map(lambda x: x['at'].strftime("%Y-%m-%d"), result[0]))
df['dateYear'] = list(map(lambda x: x['at'].year, result[0]))
df['dateMonth'] = list(map(lambda x: x['at'].month, result[0]))
df['dateDay'] = list(map(lambda x: x['at'].day, result[0]))
df['rating'] = list(map(lambda x: x['score'], result[0]))
df['userName'] = list(map(lambda x: x['userName'], result[0]))
df['content'] = list(map(lambda x: x['content'], result[0]))
df['dateYearMonth'] = list(map(lambda x: x['at'].strftime("%Y-%m"), result[0]))

In [134]:
df.head()

,date,dateYear,dateMonth,dateDay,rating,userName,content,dateYearMonth
0,2022-04-18,2022,4,18,5,김흥서,[ W009474 ] 입출금알림을 원뱅킹앱으로 사용하면 연속적으로 출금이 됐을 때 ...,2022-04
1,2022-04-17,2022,4,17,5,so young kim,[ W009474 ] 확실히 간편해졌네요. 상품 가입도 쉽게 할 수 있어보이네요. ...,2022-04
2,2022-04-27,2022,4,27,1,이정신,우리은행에서 발급한 인증서를 가져오기해서 등록했지만 새로 로그인할때마다 등록된 인증...,2022-04
3,2022-03-11,2022,3,11,1,이소현,잘 될때는 잘 되지만 한 번 에러 발생하면 진짜 30분을 넘게 결제를 반복해서 진행...,2022-03
4,2022-03-03,2022,3,3,1,박경한,신분증 촬영이 안되네요. 초점이 안맞질않나 초점이 맞아서 제대로 맞춰도 아무반응이없...,2022-03


In [ ]:
################################################### 저장할 위치 바꾸기
df.to_csv('C:/Users/yello/mba/wooribank_review_dataset.csv', encoding = 'utf-8-sig', index=False) # csv 파일로 저장

In [119]:
# result_kakao = reviews(
#     'com.kakaobank.channel',    
#     lang='ko',
#     country='kr',
#     sort=Sort.MOST_RELEVANT,
#     count=20000,
# )

In [120]:
# print(len(result_kakao[0]))

17886
